# 00 - Environment and Conventions

This notebook initializes the environment, defines conventions, and runs a smoke test comparing Black-Scholes and Monte Carlo pricing.

In [1]:
# Colab setup
import sys

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    get_ipython().system('pip install -q numpy pandas scipy matplotlib ipywidgets pytest')
    # Optional data package
    get_ipython().system('pip install -q yfinance')
    from google.colab import output
    output.enable_custom_widget_manager()
else:
    print('Local runtime detected; skip Colab-only installation cell.')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.2 MB/s eta 0:00:00


In [15]:
from pathlib import Path
import sys
import shutil


REPO_NAME = 'Interactive_Portfolio_Optimization'
REPO_URL = 'https://github.com/basarr/interactive_portfolio_optimization'


repo_path = Path('/content') / REPO_NAME

# sanity check
if repo_path.exists():
    print(f"Removing existing repository '{REPO_NAME}' at {repo_path}...")
    shutil.rmtree(repo_path)
    print(f"Existing repository '{REPO_NAME}' removed.")

print(f"Cloning '{REPO_NAME}' repository from {REPO_URL}...")
try:

    get_ipython().system(f'git clone {REPO_URL} "{repo_path}"')
    print(f"Repository '{REPO_NAME}' cloned successfully to {repo_path}")
except Exception as e:
    print(f"Error cloning repository: {e}")
    print("Please ensure the REPO_URL is correct and the repository is publicly accessible or you are authenticated.")
    # Fallback
    ROOT = Path.cwd()
    if not (ROOT / 'src').exists():
        ROOT = ROOT.parent
    print(f"Falling back to original ROOT detection: {ROOT}. This might still result in ModuleNotFoundError if 'src' is not directly present.")

ROOT = repo_path

# Ensure the project root is in sys.path
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import numpy as np
import pandas as pd
import matplotlib
import scipy


from src.config import config_dict
from src.utils import ensure_dirs, set_seed
from src.black_scholes import bs_call_price
from src.monte_carlo import mc_price_european_gbm_terminal

cfg = config_dict(fast_mode=True)
set_seed(cfg['SEED'])
ensure_dirs([
    str(ROOT / cfg['RESULTS_DIR']),
    str(ROOT / cfg['FIG_DIR']),
    str(ROOT / cfg['TABLE_DIR']),
    str(ROOT / 'results' / 'logs'),
    str(ROOT / 'results' / 'reports'),
])

print('numpy', np.__version__)
print('pandas', pd.__version__)
print('scipy', scipy.__version__)
print('matplotlib', matplotlib.__version__)
print('Conventions: continuous compounding, time in years, dt=T/n_steps')

Removing existing repository 'Interactive_Portfolio_Optimization' at /content/Interactive_Portfolio_Optimization...
Existing repository 'Interactive_Portfolio_Optimization' removed.
Cloning 'Interactive_Portfolio_Optimization' repository from https://github.com/basarr/interactive_portfolio_optimization...
Cloning into '/content/Interactive_Portfolio_Optimization'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 51 (delta 7), reused 44 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 36.19 KiB | 2.78 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Repository 'Interactive_Portfolio_Optimization' cloned successfully to /content/Interactive_Portfolio_Optimization
numpy 2.0.2
pandas 2.2.2
scipy 1.16.3
matplotlib 3.10.0
Conventions: continuous compounding, time in years, dt=T/n_steps


In [13]:
bs = bs_call_price(cfg['S0'], cfg['K'], cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T'])
mc = mc_price_european_gbm_terminal(
    S0=cfg['S0'],
    K=cfg['K'],
    r=cfg['R'],
    q=cfg['Q'],
    sigma=cfg['SIGMA'],
    T=cfg['T'],
    n_paths=20_000,
    option_type='call',
    antithetic=True,
    seed=cfg['SEED'],
)

smoke = pd.DataFrame([
    {
        'bs_call_price': bs,
        'mc_price': mc['price'],
        'ci_low': mc['ci_low'],
        'ci_high': mc['ci_high'],
        'bs_inside_ci': mc['ci_low'] <= bs <= mc['ci_high'],
    }
])
out_path = ROOT / 'results' / 'tables' / 'smoke_test.csv'
smoke.to_csv(out_path, index=False)
smoke


,bs_call_price,mc_price,ci_low,ci_high,bs_inside_ci
0,8.916037,8.941028,8.747585,9.134471,True
